In [2]:
import os, sys
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
import torch
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
import cv2
from arguments import ModelParams, PipelineParams, ModelHiddenParams
from scene import Scene, GaussianModel
from segment_anything import (SamAutomaticMaskGenerator, SamPredictor, sam_model_registry)
import imageio
from utils.segment_utils import *
from scene.hyper_loader import Load_hyper_data
from scene.neural_3D_dataset_NDC import Neural3D_NDC_Dataset

%load_ext autoreload
%autoreload 2

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
datadir = "data/hypernerf/chickchicken"
test_cam_infos = Load_hyper_data(datadir, 0.5, False, split="test")
print(len(test_cam_infos))

113


In [5]:
model_type = 'vit_h'
sam = sam_model_registry[model_type](checkpoint='/data/sxj/dependencies/sam_ckpt/sam_vit_h_4b8939.pth').to('cuda')
predictor = SamPredictor(sam)

In [ ]:
index = 113

img = test_cam_infos[index].image
img = to8b(img.permute([1, 2, 0]))
input_point = np.array([[200, 200], [270, 380], [310, 450]])
# input_point = np.array([[200, 300]])``
# input_point = np.array([[200, 300]])
# input_label = np.ones(len(input_point))
input_label = np.array([1, 0, 1])

with torch.no_grad():
    predictor.set_image(img)
    vanilla_masks, scores, logits = predictor.predict(
        point_coords=input_point,
        point_labels=input_label,
        multimask_output=True,
    )
l = len(vanilla_masks)
mask_id = np.argmax(scores)
plt.imshow(img)
show_points(input_point, input_label, plt.gca())
show_mask(vanilla_masks[mask_id], plt.gca())
plt.title(f"Mask {mask_id}, Score: {scores[mask_id]:.3f}", fontsize=18)
# plt.axis('off')
plt.show()

save_dir = os.path.join(datadir, "gt_mask", "egg")
os.makedirs(save_dir, exist_ok=True)
cv2.imwrite(os.path.join(save_dir, test_cam_infos[index].image_name), vanilla_masks[mask_id].astype("int"))